# Diluted Unitary Fitting with KL-Div

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from quantum_tools import generate_unitary
from quantum_channel import channel_spectrum
from loss_functions import ProbabilityMSE, channel_fidelity_loss, SpectrumDistance
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap, DilutedKrausMap
from lindblad_channels import CompactLindbladMap
from spam import InitialState, POVM, SPAM, CorruptionMatrix
from utils import saver
from synthetic_data import generate_spam_data, generate_spam_benchmark, generate_map_data
from spectrum import mean_spacing
from utils import loader

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = SpectrumDistance(sigma=sigma, k = None), 
                                            verbose=verbose,
                                            N=0),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 3:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Four Qubits, Repetition

In [3]:
n = 4
d = 2**n

path1 = "../../../data/"
path2 = "data/"

[model1, model2, model3, model4, model5] = loader(path1 + "belem_4qubit_repeated.model")

spectrum1 = channel_spectrum(model1.channel, keep_unity=False)
spectrum2 = channel_spectrum(model2.channel, keep_unity=False) 
spectrum3 = channel_spectrum(model3.channel, keep_unity=False)
spectrum4 = channel_spectrum(model4.channel, keep_unity=False)
spectrum5 = channel_spectrum(model5.channel, keep_unity=False)

In [11]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.02431698265030642 None
-0.014457176579201224 None
-0.01445795505108801 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.019621132351829877 None
-0.02051658083907932 None
-0.020516604866696403 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.02251986109199975 None
-0.02259530925022129 None
-0.022595317115734447 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.021161286123043556 None
-0.021202332344770668 None
-0.021202333253125774 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.019044643485042777 None
-0.019140033829715818 None
-0.019140035811805317 None
Done!


In [12]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 10, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_16layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.04533811068121134 None
0.045329996617866984 None
0.0453299964368201 None


  0%|          | 0/200 [00:00<?, ?it/s]

0.048500407705357323 None
-0.004775618224579954 None
-0.0050628142558500275 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024810470474832036 None
-0.024823852118013703 None
-0.02482385386643443 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02270932018871931 None
-0.023362337724488737 None
-0.023362361145985916 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.022285456585814212 None
-0.022615400341474218 None
-0.022615407816214343 None
Done!


In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = 0.18,
                                                      rank_start = 40, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_24layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

-0.009984272049604045 None
-0.015264161447782622 None
-0.015264171334487528 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.013456041423519222 None
-0.015784388754267388 None
-0.01578442147142492 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.024435826185256602 None
-0.024453200951644452 None
-0.024453204241121615 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.02265450407977239 None
-0.022681747009569493 None
-0.02268174712055275 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.021485065906558506 None
-0.02296371545447011 None
-0.02296383567084083 None
Done!


In [4]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = 0.10, 
                                                      rank_start = 100, 
                                                      rank_step = 1,
                                                      num_iter = 300,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_32layer.model")

  0%|          | 0/300 [00:00<?, ?it/s]

-0.0387515613375851 None
-0.044158180869354566 None
-0.04415835398119773 None
-0.044158353987078056 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.039528217774079596 None
-0.03959038252774055 None
-0.03959038386479008 None
-0.03959038386481625 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.045519727254834066 None
-0.047226098402719796 None
-0.04722614425412841 None
-0.04722614425493875 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.039181588991877375 None
-0.039238876424631544 None
-0.0392388806275253 None
-0.039238880627655875 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03617806085696132 None
-0.03622800348135724 None
-0.03622800463972817 None
-0.03622800463982859 None
Done!


In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum5, 
                                                      d = d,
                                                      c_start = 0.10, 
                                                      rank_start = 100, 
                                                      rank_step = 1,
                                                      num_iter = 300,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_40layer.model")

  0%|          | 0/300 [00:00<?, ?it/s]

-0.004737941247782984 None
-0.03787279979919901 None
-0.0378798164529017 None
-0.03787981649660596 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03970895229089666 None
-0.039711740827375566 None
-0.0397117430677994 None
-0.039711743067856534 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03541919551430274 None
-0.03545701244014264 None
-0.03545701322359257 None
-0.03545701322362371 None


  0%|          | 0/300 [00:00<?, ?it/s]

-0.03778877467933171 None
-0.03797028674791407 None
-0.037970298980582004 None
-0.03797029898058932 None
Done!


## Four Qubits, Delay

In [18]:
n = 4
d = 2**n

path1 = "../../../data/"
path2 = "data/"

[model1, model2, model3, model4, model5, model6, model7, model8, model9] = loader(path1 + "belem_4qubit_delay.model")

spectrum1 = channel_spectrum(model1.channel, keep_unity=False)
spectrum2 = channel_spectrum(model2.channel, keep_unity=False) 
spectrum3 = channel_spectrum(model4.channel, keep_unity=False)
spectrum4 = channel_spectrum(model6.channel, keep_unity=False)

In [13]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d,
                                                      c_start = 0.4,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_0delay.model")

0.014205054440732518 None
0.008845493580107968 None
0.0041204841226500505 None
0.005524199912069497 None
0.005178203361651448 None
0.005439743360235921 None
0.005972243073423356 None
0.008965927547632122 None
Done!


In [15]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = 0.3,
                                                      rank_start = 10, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_5delay.model")

0.01848348460790743 None
0.014186931516701865 None
0.009715298239793096 None
0.007303294242447408 None
0.005723892968955122 None
0.004077427925329443 None
0.0025232176524793357 None
0.001863246488379957 None
0.003712720919869558 None
0.003806595261150243 None
0.0063913662393518395 None
0.006416087967554447 None
0.009325553550093457 None
Done!


In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = 0.18,
                                                      rank_start = 45, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_10delay.model")

0.009643465250119252 None
0.010842895343623385 None
0.012202345632732953 None
0.011250103925621733 None
0.011827404001205143 None
0.01217502180153485 None
Done!


In [19]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = 0.12, 
                                                      rank_start = 65, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_12_5delay.model")

0.012446957367763904 None
0.012270846412483716 None
0.01264303974927565 None
0.012950804567063212 None
0.012072362215487242 None
0.013101705753887218 None
0.013578943158130453 None
0.013536197943886753 None
Done!
